In [1]:
import requests
import requests_cache
from bs4 import BeautifulSoup
import time
import json
import re
import pandas as pd # Dùng để đọc và hiển thị file csv/tsv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from requests_html import HTML

________________________________________

In [3]:
# Lưu user
users = []
# Lưu track
tracks = {"track":[], "user":[]}
# Lưu playlist
playlists = []
# Lưu link playlist của producer
urls_user_playlist = []
# Lưu link playlist của discover
urls_playlist = []

**Chuyển từ list sang csv/tsv**

In [4]:
def list_to_tsv_csv(listItem, outputFile):
    df = pd.DataFrame(listItem)
    df.to_csv(outputFile, sep='\t')

**Bấm để đồng ý cookies**

In [5]:
def accept_cookies():
    soup = BeautifulSoup(browser.page_source)
    cookies = browser.find_element(By.XPATH,"//button[@id='onetrust-accept-btn-handler']")
    cookies.click()

**Bấm vào các nút > đến hết ở trang discover**

In [6]:
def click_button():
    soup = BeautifulSoup(browser.page_source)
    genres = soup.find_all("li", "mixedModularHome__item")

    for i in range(len(genres)):
        lists = genres[i].find_all("div", "tileGallery__sliderPanelSlide")
        j = 4
        while True:
            if j > len(lists):
                break
            browser.find_elements(By.XPATH,"//button[contains(.,'forward')]")[i].click()
            j += 4

**Lấy hết tất cả playlists ở trang discover**

In [7]:
def get_all_playlists_discover():
    soup = BeautifulSoup(browser.page_source)  
    genres = soup.find_all("li", "mixedModularHome__item")

    for i in range(len(genres)):
        lists = genres[i].find_all("div", "tileGallery__sliderPanelSlide")
        for pl in lists:
            title = pl.find("a", "playableTile__heading").get_text().strip()
            link = pl.find("a", "playableTile__artworkLink")["href"]
            if title not in playlists:
                playlists.append(title)
                urls_playlist.append("https://soundcloud.com" + link)

**Kéo xuống đến hết trang**

In [25]:
def scroll_down():
    soup = BeautifulSoup(browser.page_source)
    
    count = int(soup.find("div", "genericTrackCount__title").get_text())
    
    i = 15
    while i < count:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        time.sleep(3)
        i += 15

**Lấy tất cả users và tracks của một playlist**

In [33]:
def get_all_users_tracks_playlist():
    scroll_down()
    
    soup = BeautifulSoup(browser.page_source)
    search = soup.find_all("li", "systemPlaylistTrackList__item")
    if len(search) == 0:
        search = soup.find_all("li", "trackList__item")

    # Trích xuất user và track trong 1 playlist
    for producer in search:
        user = producer.find("a", "trackItem__username").get_text()
        url = producer.find("a", "trackItem__username").get('href')
        track = producer.find("a", "trackItem__trackTitle").get_text()
        if user not in users:
            users.append(user)
            urls_user_playlist.append("https://soundcloud.com" + url + "/sets")
            print(user)
        if track not in tracks["track"]:
            tracks["track"].append(track)
            tracks["user"].append(user)
            print(track)

**Lấy tất cả playlists của một user**

In [23]:
def get_all_playlists_user():
    soup = BeautifulSoup(browser.page_source)
    posts = soup.find_all("li", "soundList__item")
    
    # kiểm tra có playlist hay không, nếu có chạy dưới, nếu không bỏ qua
    if len(posts) != 0:
        # nếu số playlists lớn hơn 4 thì sẽ scroll
        if len(posts) > 4:
            scroll_down()
            soup = BeautifulSoup(browser.page_source)
            
        # trích xuất playlist
        for post in posts:
            title = post.find("a", "soundTitle__title").get_text().strip()
            if title not in playlists:
                playlists.append(title)
                browser.find_elements(By.XPATH,"//a[@class='compactTrackList__moreLink']")[i].click()
                

In [37]:
def parse_users_tracks_per_genre():
    for url in urls_playlist:
        browser.execute_script('window.open("{link}");'.format(link = url));
        browser.switch_to.window(browser.window_handles[1])
        time.sleep(2)
        get_all_users_tracks_playlist()
        time.sleep(1)
        browser.close()
        browser.switch_to.window(browser.window_handles[0])

In [12]:
def parse_playlists_per_genre():
    for url in urls_user_playlist:
        browser.execute_script('window.open("{link}");'.format(link = url));
        browser.switch_to.window(browser.window_handles[1])
        time.sleep(2)
        get_all_playlists_user()
        time.sleep(1)
        browser.close()
        browser.switch_to.window(browser.window_handles[0])

In [26]:
browser = webdriver.Chrome()
browser.get("https://soundcloud.com/discover")

In [28]:
accept_cookies()

In [29]:
click_button()
get_all_playlists_discover()

In [38]:
parse_users_tracks_per_genre()

Major Lazer - Powerful feat. Ellie Goulding & Tarrus Riley (Michael Calfan Remix)
Cazzette - Together (Lost Kings Remix)
Tritonal & Cash Cash - Untouchable (BRKLYN Remix)
Axwell Λ Ingrosso - Sun Is Shining (R3hab Remix)
KSHMR, Dzeko & Torres - Imaginate (Original Mix) [Free Download]
BL3R - Heat (Original Mix)
DVBBS - Raveheart (Out Now)
Alvita - Empire (Radio Edit)
Deorro x MAKJ - Ante Up
Marshmello - Moving On (Original Mix)
Local Natives
Dark Days (Sofi Tukker Remix)
Here Comes The Night (Junkie Kid Remix) [feat. Mr Hudson]
The Chainsmokers & Coldplay - Something Just Like This (Koni Remix ft.  Marina Lin)
White Panda
White Panda - Save Dat One For Me (Lil Dicky ft. Fetty Wap // Great Good Fine Ok)
Bruno Torres Remixes
Sia - Cheap Thrills Ft. Sean Paul (Bruno Torres Remix)
Axwell Λ Ingrosso
Something New (Robin Schulz Remix)
Brooks
Martin Garrix & Dua Lipa - Scared To Be Lonely (Brooks Remix)
Jay Nebula
5am (feat. Joey XL)
Lick Twist - Love Better (feat. Michele Wylen)
HoMe (Origina

larrysklai
Spa SOOTHING SOUNDS Relaxing Music - Spring Morning
nithoukhan
Relaxing Soothing Rabaab Instrumental (Amir Mohammad)
user301749422
2 Hour Shamanic Tibetan Meditation  Calming Music, Sooting Music, Deep Meditation Music ☯181
soundcanyon
Windfall
Jack Villano Villano
Mozart - Requiem in D minor Complete Full
Root Chakra Healing Frequency 432 Hz Meditation Music - Powerful Relaxing Music
Joe Med 1
Airstream - Electra - The Most Chillout Tune Ever 2
Arturix Panther
Positive Thinking  Relaxation Meditation Music,Relaxing Nature Sounds, Zen Meditation,Massage Music
Better Off Without You
Max Frost
Let Me Down Easy
Ooh La La Records
Ace Reporter - Collected Works
Daria Odinokova
Erlend Øye - Garota
Acacia
Korby Lenker
Got to Be More
RHODES
RHODES + BIRDY - Let It All Go
Low - Lies
Indica Records
Half Moon Run - Full Circle
justapoco
Pillow Talk - Wild Child
bearmachine
Radical Face - Wrapped In Piano Strings
Foals - London Thunder [Lakechild Edit]
Carolx
City and Colour - Comin' Ho

In [ ]:
browser.quit()

In [50]:
# nên check lại bởi vì vài lần chưa scroll hết :<
def scroll_down2():
    soup = BeautifulSoup(browser.page_source)
    
    for i in range(4):
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        time.sleep(3)

In [45]:
browser.quit()

browser = webdriver.Chrome()
browser.get("https://soundcloud.com/discover")
time.sleep(5)
accept_cookies()

parse_playlists_per_genre()

KeyboardInterrupt: 

**Lưu data vào file tsv**

In [39]:
outputFile = "Crawl_data/user.csv"
list_to_tsv_csv(users, outputFile)
outputFile = "Crawl_data/track.csv"
list_to_tsv_csv(tracks, outputFile)
outputFile = "Crawl_data/playlist.csv"
list_to_tsv_csv(playlists, outputFile)

In [53]:
print(len(users))
print(len(tracks["track"]))
print(len(playlists))

5632
8167
305
